--- Day 8: Resonant Collinearity ---

You find yourselves on the roof of a top-secret Easter Bunny installation.

While The Historians do their thing, you take a look at the familiar huge antenna. Much to your surprise, it seems to have been reconfigured to emit a signal that makes people 0.1% more likely to buy Easter Bunny brand Imitation Mediocre Chocolate as a Christmas gift! Unthinkable!

Scanning across the city, you find that there are actually many such antennas. Each antenna is tuned to a specific frequency indicated by a single lowercase letter, uppercase letter, or digit. You create a map (your puzzle input) of these antennas. For example:
```
............
........0...
.....0......
.......0....
....0.......
......A.....
............
............
........A...
.........A..
............
............
```
The signal only applies its nefarious effect at specific antinodes based on the resonant frequencies of the antennas. In particular, an antinode occurs at any point that is perfectly in line with two antennas of the same frequency - but only when one of the antennas is twice as far away as the other. This means that for any pair of antennas with the same frequency, there are two antinodes, one on either side of them.

So, for these two antennas with frequency a, they create the two antinodes marked with #:
```
..........
...#......
..........
....a.....
..........
.....a....
..........
......#...
..........
..........
```
Adding a third antenna with the same frequency creates several more antinodes. It would ideally add four antinodes, but two are off the right side of the map, so instead it adds only two:
```
..........
...#......
#.........
....a.....
........a.
.....a....
..#.......
......#...
..........
..........
```
Antennas with different frequencies don't create antinodes; A and a count as different frequencies. However, antinodes can occur at locations that contain antennas. In this diagram, the lone antenna with frequency capital A creates no antinodes but has a lowercase-a-frequency antinode at its location:
```
..........
...#......
#.........
....a.....
........a.
.....a....
..#.......
......A...
..........
..........
```
The first example has antennas with two different frequencies, so the antinodes they create look like this, plus an antinode overlapping the topmost A-frequency antenna:
```
......#....#
...#....0...
....#0....#.
..#....0....
....0....#..
.#....A.....
...#........
#......#....
........A...
.........A..
..........#.
..........#.
```
Because the topmost A-frequency antenna overlaps with a 0-frequency antinode, there are 14 total unique locations that contain an antinode within the bounds of the map.

Calculate the impact of the signal. How many unique locations within the bounds of the map contain an antinode?

In [1]:
import numpy as np
from collections import defaultdict

In [2]:
with open('input.txt', 'r') as f:
    
    file = f.read().splitlines()

data = np.array([[i for i in line] for line in file])

In [3]:
def get_positions(data):
    
    positions = defaultdict(list)
    
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            if data[i, j] != '.':
                positions[data[i, j]].append((i, j))
                
    return positions

In [4]:
def find_antinodes(position, data_shape):
    
    antinodes = set()
    
    for i in range(len(position)):
        for j in range(i + 1, len(position)):
            
            a1 = tuple(position[i])
            a2 = tuple(position[j])

            vec = (a2[0] - a1[0], a2[1] - a1[1])
            
            ant1 = (a2[0] + vec[0], a2[1] + vec[1])
            ant2 = (a1[0] - vec[0], a1[1] - vec[1])
            
            if 0 <= ant1[0] < data_shape[0] and 0 <= ant1[1] < data_shape[1]:
                antinodes.add(ant1)
                
            if 0 <= ant2[0] < data_shape[0] and 0 <= ant2[1] < data_shape[1]:
                antinodes.add(ant2)
                
    return antinodes

In [5]:
positions = get_positions(data)
all_antinodes = set()

for antenna, position in positions.items():
    
    antinodes = find_antinodes(position, data.shape)
    all_antinodes.update(antinodes)

In [6]:
len(all_antinodes)

364

That's the right answer! You are one gold star closer to finding the Chief Historian.

--- Part Two ---
    
Watching over your shoulder as you work, one of The Historians asks if you took the effects of resonant harmonics into your calculations.

Whoops!

After updating your model, it turns out that an antinode occurs at any grid position exactly in line with at least two antennas of the same frequency, regardless of distance. This means that some of the new antinodes will occur at the position of each antenna (unless that antenna is the only one of its frequency).

So, these three T-frequency antennas now create many antinodes:
```
T....#....
...T......
.T....#...
.........#
..#.......
..........
...#......
..........
....#.....
..........
```
In fact, the three T-frequency antennas are all exactly in line with two antennas, so they are all also antinodes! This brings the total number of antinodes in the above example to 9.

The original example now has 34 antinodes, including the antinodes that appear on every antenna:
```
##....#....#
.#.#....0...
..#.#0....#.
..##...0....
....0....#..
.#...#A....#
...#..#.....
#....#.#....
..#.....A...
....#....A..
.#........#.
...#......##
```
Calculate the impact of the signal using this updated model. How many unique locations within the bounds of the map contain an antinode?

In [7]:
def find_antinodes_with_resonance(position, data_shape):
    
    antinodes = set()
    
    for i in range(len(position)):
        for j in range(i + 1, len(position)):
            
            a1 = tuple(position[i])
            a2 = tuple(position[j])

            vec = (a2[0] - a1[0], a2[1] - a1[1])
            
            ant1 = (a2[0] + vec[0], a2[1] + vec[1])
            ant2 = (a1[0] - vec[0], a1[1] - vec[1])

            step = np.gcd(vec[0], vec[1])
            unit_vec = (vec[0] // step, vec[1] // step)
            
            temp = a2
            while 0 <= temp[0] < data_shape[0] and 0 <= temp[1] < data_shape[1]:
                antinodes.add(temp)
                temp = (temp[0] + unit_vec[0], temp[1] + unit_vec[1])

            temp = a1
            while 0 <= temp[0] < data_shape[0] and 0 <= temp[1] < data_shape[1]:
                antinodes.add(temp)
                temp = (temp[0] - unit_vec[0], temp[1] - unit_vec[1])

    return antinodes

In [8]:
all_antinodes_part_two = set()

for freq, pos in positions.items():
    freq_antinodes = find_antinodes_with_resonance(pos, data.shape)
    all_antinodes_part_two.update(freq_antinodes)

In [9]:
len(all_antinodes_part_two)

1231

That's the right answer! You are one gold star closer to finding the Chief Historian.